# Import dependencies

In [1]:
import sys
import re
from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from collections import Counter

sys.path.append("../../.")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *
from calculus_path_mod.terms import num_comparison, nat_numbers, arithm_ops, combinators, pairs, logic
from calculus_path_mod.terms.pseudonym import *

from contextlib import contextmanager
import threading
import _thread

from calculus_path_mod.json_serialization import load_terms

In [2]:
class TimeoutException(Exception):
    def __init__(self, msg=''):
        self.msg = msg


@contextmanager
def time_limit(seconds, msg=''):
    timer = threading.Timer(seconds, lambda: _thread.interrupt_main())
    timer.start()
    try:
        yield
    except KeyboardInterrupt:
        raise TimeoutException("Timed out for operation {}".format(msg))
    finally:
        # if the action ends in specified time, the timer is canceled
        timer.cancel()

# Do generation

In [8]:
# from_inx, to_inx = 0, 20
# #
# print(from_inx, "-", to_inx)
# lists_terms_LO = [load_terms(f"../tests_11_retests/collected_terms/terms_210_filtered_LO_{inx_}.dat") for inx_ in range(from_inx, to_inx)]

lists_terms_LO = [
    load_terms(f"../tests_11_retests/terms_210_filtered_LO.dat"),
    load_terms(f"../tests_11_retests/terms_210_filtered_RI.dat"),
]

In [10]:
def gen_norm_data(terms_list, strategy):
    term_reductions_list = []

    for term in tqdm(terms_list):
        norm_term = term
        term_red_step = 0
        reductions = []
        while norm_term:
            term_red_step += 1
            reductions.append(norm_term.funky_str())
            (step_term, _, _), norm_term = norm_term.one_step_normalize_visual(strategy)
            if (step_term.vertices_number > 1_000) or (term_red_step > 30):
                norm_term = None
        term_reductions_list.append(reductions)

    return term_reductions_list

In [11]:
list_res_OO = [gen_norm_data(terms_, LOStrategy()) for terms_ in lists_terms_LO]

100%|██████████| 223/223 [00:01<00:00, 193.42it/s]


In [19]:
trs = {"term": [], "term_next_LO": []}

for term_reductions_list_ in list_res_OO:
    for term_reductions_ in term_reductions_list_:
        for inx in range(len(term_reductions_) - 1):
            trs['term'].append(term_reductions_[inx])
            trs['term_next_LO'].append(term_reductions_[inx + 1])

In [20]:
df = pd.DataFrame(trs)
print(len(df))
df = df.drop_duplicates(subset="term")
print(len(df))

3457
3416


In [21]:
df.head()

,term,term_next_LO
0,((λx.(λy.((y y) (λz.z)))) (λa.(((λb.b) (a (λc....,(λx.((x x) (λy.y)))
1,(((λx.(λy.((y y) (λz.((λa.(λb.(λc.(x (c (a c))...,((λx.((x x) (λy.((λz.(λa.(λb.((λc.(((λd.(c d))...
2,((λx.((x x) (λy.((λz.(λa.(λb.((λc.(((λd.(c d))...,((((λx.x) ((λy.y) (λz.(z (λa.z))))) ((λx.x) ((...
3,((((λx.x) ((λy.y) (λz.(z (λa.z))))) ((λx.x) ((...,((((λx.x) (λy.(y (λz.y)))) ((λa.a) ((λb.b) (λc...
4,((((λx.x) (λy.(y (λz.y)))) ((λa.a) ((λb.b) (λc...,(((λx.(x (λy.x))) ((λz.z) ((λa.a) (λb.(b (λc.b...


In [22]:
df.to_csv(f"./data/term_step_LO.csv", index=False)